In [ ]:
import pandas as pd
import pyodbc 
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy as sal
from sqlalchemy import create_engine

In [ ]:
#Connect to Data Structure for sqlalchemy is: "mssql+pyodbc://OpR-Marc-DB2:1433/RPC1?driver=ODBC+Driver+17+for+SQL+Server" --change RPC1 for other volumes

RPC1 = sal.create_engine("mssql+pyodbc://OpR-Marc-DB2:1433/RPC1?driver=ODBC+Driver+17+for+SQL+Server")
connection = RPC1.connect()

RC1 = sal.create_engine("mssql+pyodbc://OpR-Marc-DB2:1433/RC1?driver=ODBC+Driver+17+for+SQL+Server")
connection = RC1.connect()


In [ ]:
RPC1AiiGJPartnersDF = pd.DataFrame()
RC1AiiGJPartnersDF = pd.DataFrame()

In [ ]:
RC1Aiis= ["514", "3679", "2610", "410"]
RPC1Aiis = ["192", "262", "265", "1685", "2710"]

In [ ]:
#Pull Data from RPC1

for x in RPC1Aiis: 
    query = "SET NOCOUNT ON; EXEC RLPfeiffer.GJPartners @TargetCell= " + "'" + x + "'"
    GJ = pd.read_sql(query, con = RPC1)
    GJ['Volume']= 'RPC1'
    GJ['AiiQueried']=x
    RPC1AiiGJPartnersDF = RPC1AiiGJPartnersDF.append(GJ)


RPC1AiiGJPartnersDF

In [ ]:
#Pull Data from RC1
for x in RC1Aiis: 
    query = "SET NOCOUNT ON; EXEC RLPfeiffer.GJPartners @TargetCell= " + "'" + x + "'"
    GJ = pd.read_sql(query, con = RC1)
    GJ['Volume']= 'RC1'
    GJ['AiiQueried']=x
    RC1AiiGJPartnersDF = RC1AiiGJPartnersDF.append(GJ)


RC1AiiGJPartnersDF

In [ ]:
#concatinate the 2 datasets
frames = [RPC1AiiGJPartnersDF, RC1AiiGJPartnersDF]
AiiGJPartnersDF = pd.concat(frames).reset_index()

In [ ]:
df = AiiGJPartnersDF
df['SourceParentLabel']=df['SourceParentLabel'].str.strip('?()wf3456n-')
df['TargetParentLabel']=df['TargetParentLabel'].str.strip('?()wf3456n-')

In [ ]:
AiiGJPartnersDF.fillna('Unk', inplace=True)

In [ ]:
df = AiiGJPartnersDF
df['GJMotif']= df['SourceParentLabel'] + '::' + df['TargetParentLabel']

In [ ]:
AiiGJPartnersDF = AiiGJPartnersDF.drop(columns=['Notes'])

In [ ]:
#These are all of the motif variations, not super clean

AiiGJPartnersDF.replace('Aii::GAC Aii','GAC Aii::GAC Aii', inplace=True)
AiiGJPartnersDF.replace('CBb::GAC Aii','GAC Aii::CBb', inplace=True)
AiiGJPartnersDF.replace('RodBC::GAC Aii','GAC Aii::RodBC', inplace=True)
AiiGJPartnersDF.replace('BC::GAC Aii','GAC Aii::BC', inplace=True)
AiiGJPartnersDF.replace('CB::GAC Aii','GAC Aii::CB', inplace=True)
AiiGJPartnersDF.replace('::GAC Aii','GAC Aii::Unk', inplace=True)
AiiGJPartnersDF.replace('Unk::GAC Aii','GAC Aii::Unk', inplace=True)
AiiGJPartnersDF.replace('GAC Aii::','GAC Aii::Unk', inplace=True)
AiiGJPartnersDF.replace('unk::GAC Aii','GAC Aii::Unk', inplace=True)
AiiGJPartnersDF.replace('GAC Aii::ull','GAC Aii::Unk', inplace=True)
AiiGJPartnersDF.replace('NULL::GAC Aii','GAC Aii::Unk', inplace=True)
AiiGJPartnersDF.replace('AC::GAC Aii','GAC Aii::AC', inplace=True)
AiiGJPartnersDF.replace('yAC::GAC Aii','GAC Aii::yAC', inplace=True)
AiiGJPartnersDF.replace('CBa::GAC Aii','GAC Aii::CBa', inplace=True)

In [ ]:
# Test of query cleaness, don't need to run this everytime
df = AiiGJPartnersDF
duplicates = df[df.duplicated(subset=['SourceID','AiiQueried'])]

duplicates

In [ ]:
df = AiiGJPartnersDF

AiiGJNoDuplicates= pd.DataFrame(df.drop_duplicates(subset=['SourceID','AiiQueried']).reset_index())

df

In [ ]:
AiiGJNoDuplicates

In [ ]:
AiiGJNoDuplicates.AiiQueried.value_counts()

In [ ]:
PartnersTotal = AiiGJPartnersDF.groupby(["AiiQueried", "GJMotif"])['SourceParent'].value_counts()
print(PartnersTotal)

In [ ]:
AiiGJNoDuplicates.GJMotif.value_counts()

In [ ]:
# Assuming AiiGJPartnersDF is your DataFrame
InputLabel = 'RodBC'

# Filter the DataFrame based on the conditions using query
filtered_df = AiiGJPartnersDF.query("AiiQueried == '192' and (SourceParentLabel == @InputLabel or TargetParentLabel == @InputLabel)")

# Count the number of unique Source Parents using value_counts
unique_source_parents = filtered_df['SourceParent'].value_counts().shape[0]

# Print the result
print("RodBCs connected:", unique_source_parents)



In [ ]:
AiiGJNoDuplicates.loc[AiiGJNoDuplicates['GJMotif'] == 'GAC Aii::AC']

In [ ]:
SimpAiiGJPartnerDF = AiiGJPartnersDF.groupby(["AiiQueried", 'Volume'])["GJMotif"].value_counts().to_frame(name = 'GJCount').reset_index()
SimpAiiGJPartnerDF

In [ ]:
df = SimpAiiGJPartnerDF
df["TotalGJs"] = df.groupby('AiiQueried')["GJCount"].transform('sum')
df["PercentofInput"] = df["GJCount"] / df["TotalGJs"]*100

In [ ]:
SimpAiiGJPartnerDF

In [ ]:

mean_std_df = SimpAiiGJPartnerDF.groupby(['Volume', 'GJMotif'])['PercentofInput'].agg(['mean', 'std'])
print(mean_std_df)

In [ ]:
AiiGJNoDuplicates['GJArea'] = AiiGJNoDuplicates['GJArea']*1.0e-6

In [ ]:
#Use this if/when you want to export something to an excel file
Aii_GJ = AiiGJNoDuplicates.to_excel('D:\Documents\Data\RPC1\Aii_GAC_Project\dfExports\AiiGJPartners.xlsx')